# INSPECT THE SYSTEM

In [22]:
import simtk.openmm as mm          # Main OpenMM functionality
import simtk.openmm.app as app     # Application layer (handy interface)
import simtk.unit as unit          # Unit/quantity handling
import os

### (a) Topology of the protein

In [23]:
cwd = os.getcwd()                                      # Current working directory path
pdb_file = os.path.join(cwd, 'files/6a5j_protein/6a5j.pdb')     # pdb file path

molecule = app.PDBFile(pdb_file)                       # To access molecule.topology
molecule.topology                                      # One can also visulaize the molecule in VMD

<Topology; 1 chains, 13 residues, 260 atoms, 259 bonds>

### (b) Molecular strucutre details

In [24]:
with open(pdb_file) as pdbfile:
    print("The structure contains:")
    atom_count = 0      # How many atoms?
    model_count = 0     # How many frames?
    for line in pdbfile:
        if line.startswith("SEQRES"):
            print(f"    {line.split()[3]} residues:")     # residues = amino acids
            print(f"        {' '.join(line.split()[4:])}\n")
            continue
        if line.startswith("MODEL"):
            model_count += 1
            atom_count = 0
            continue
        if line.startswith("ATOM"):
            atom_count += 1
            continue
    print(f"    {atom_count} atoms\n")
    print(f"    {model_count} frames\n")     # frames = structures

The structure contains:
    13 residues:
        ILE LYS LYS ILE LEU SER LYS ILE LYS LYS LEU LEU LYS

    260 atoms

    20 frames



### (c) Charge of the molecule

In [ ]:
# (i) Rational approach: The peptide has 6 lysine residues in its sequence. 
#                        All lysine side chains are protonated and the total 
#                        charge of the molecule is +6.

In [25]:
# (ii) Forcefield approach:

forcefield = app.ForceField("amber14/protein.ff14SB.xml")
system = forcefield.createSystem(molecule.topology)

In [26]:
for force in system.getForces():
    print(type(force))
    if isinstance(force, mm.openmm.NonbondedForce):
        # For charges, we are only interested in the non-bonded forces
        nonbonded = force

<class 'simtk.openmm.openmm.HarmonicBondForce'>
<class 'simtk.openmm.openmm.HarmonicAngleForce'>
<class 'simtk.openmm.openmm.PeriodicTorsionForce'>
<class 'simtk.openmm.openmm.NonbondedForce'>
<class 'simtk.openmm.openmm.CMMotionRemover'>


In [27]:
nonbonded.getParticleParameters(0)     # Non-bonded force entry of an atom

[Quantity(value=0.0311, unit=elementary charge),
 Quantity(value=0.3249998523775958, unit=nanometer),
 Quantity(value=0.7112800000000001, unit=kilojoule/mole)]

In [28]:
force_description = zip(
    ["Partial charge", "Minimum energy distance", "Minimum energy"],
    nonbonded.getParticleParameters(0)
)

for desc, comp in force_description:
    print(f"{desc:<26} {comp}")

Partial charge             0.0311 e
Minimum energy distance    0.3249998523775958 nm
Minimum energy             0.7112800000000001 kJ/mol


In [32]:
total_charge = 0.
for i in range(nonbonded.getNumParticles()):
    nb_i = nonbonded.getParticleParameters(i)      
    total_charge += nb_i[0].value_in_unit(unit.elementary_charge)

total_charge

5.999999999999995

In [33]:
round(total_charge)

6